In [1]:
import numpy as np
import networkx as nx
from qat.opt import GraphColouring
import qat
from copy import deepcopy

In [2]:
base  = 3
side  = base*base

# pattern for a baseline valid solution
def pattern(r,c): return (base*(r%base)+r//base+c)%side

# randomize rows, columns and numbers (of valid base pattern)
from random import sample
def shuffle(s): return sample(s,len(s)) 
rBase = range(base) 
rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
nums  = shuffle(range(1,base*base+1))

# produce board using randomized baseline pattern
board = [ [nums[pattern(r,c)] for c in cols] for r in rows ]

for line in board: print(line)

[1, 6, 8, 3, 5, 2, 7, 9, 4]
[7, 4, 9, 1, 6, 8, 3, 2, 5]
[3, 5, 2, 7, 4, 9, 1, 8, 6]
[9, 7, 6, 8, 1, 5, 2, 4, 3]
[8, 1, 5, 2, 3, 4, 9, 6, 7]
[2, 3, 4, 9, 7, 6, 8, 5, 1]
[6, 9, 1, 5, 8, 3, 4, 7, 2]
[5, 8, 3, 4, 2, 7, 6, 1, 9]
[4, 2, 7, 6, 9, 1, 5, 3, 8]


In [3]:
np.savetxt('board.ans', board, delimiter=' ', fmt='%i')

In [41]:
squares = side*side
empties = 30
board_em = deepcopy(board)
for p in sample(range(squares),empties):
    board_em[p//side][p%side] = 0

numSize = len(str(side))
for line in board_em: print("["+"  ".join(f"{n or '.':{numSize}}" for n in line)+"]")

[1  .  8  3  5  2  .  9  .]
[.  4  .  1  6  8  3  2  .]
[3  5  .  7  .  9  .  8  6]
[.  7  .  8  1  .  .  .  3]
[8  1  5  2  .  4  9  .  7]
[2  3  4  .  7  6  .  .  .]
[6  .  .  .  .  3  4  7  .]
[5  8  .  4  .  7  6  .  9]
[4  2  7  6  9  1  .  3  .]


In [42]:
np.savetxt('board.in', board_em, delimiter=' ', fmt='%i')

In [30]:
number_of_colours = side

graph = nx.Graph()
graph.add_nodes_from(np.arange(side * side))

# add edge for same row
for row in range(side):
    row_start_spin = row * side
    row_end_spin = (row + 1) * side
    for spin_from in range(row_start_spin, row_end_spin):
        for spin_to in range(spin_from, row_end_spin):
            if spin_from != spin_to:
                graph.add_edges_from([(spin_from, spin_to)])

# add edge for same col
for col in range(side):
    col_start_spin = col
    col_end_spin = side * side + col
    for spin_from in range(col_start_spin, col_end_spin, side):
        for spin_to in range(spin_from, col_end_spin, side):
            if spin_from != spin_to:
                graph.add_edges_from([(spin_from, spin_to)])

# add edge for same block
for block_row in range(base):
    for block_col in range(base):
        block_spins = []
        first_spin = block_row * (base * side) + block_col * (base)
        for row_first_spin in range(first_spin, first_spin + side * base, side):
            for spin in range(row_first_spin, row_first_spin + base):
                block_spins.append(spin)
                
        for spin_from_idx in range(len(block_spins)):
            for spin_to_idx in range(spin_from_idx, len(block_spins)):
                spin_from = block_spins[spin_from_idx]
                spin_to = block_spins[spin_to_idx]
                if spin_from != spin_to:
                    graph.add_edges_from([(spin_from, spin_to)])


In [31]:
graph_colouring_problem = GraphColouring(graph, number_of_colours)

print("To anneal the problem, the solver would need "
      + str(len(graph.nodes()) * number_of_colours) + " spins.")


To anneal the problem, the solver would need 729 spins.


In [32]:
J, h, offset = graph_colouring_problem.to_ising().get_j_h_and_offset()

In [25]:
np.savetxt('J.in', J, delimiter=' ', fmt='%f')
np.savetxt('h.in', h, delimiter=' ', fmt='%f')